# テンソルネットワークのべんきょう

西野友年「テンソルネットワーク入門」講談社，2023に出てくる計算を実際にプログラムしてみます．

- 言語はJuliaで


## 第3章 畳で学ぶ転送行列

$n$畳間に畳を敷き詰める方法は何通りあるか？をvertex模型と考えて，テンソルネットワークとみなす．

---

### 4脚テンソルの値

本書では4脚テンソルの値は次のように決められている．

$$
W_{0010} = W_{1000} = W_{0001} = W_{0100} = 1,
$$
それ以外の配置の場合には
$$
    W_{abcd} = 0
$$
ただし，テキスト中ではテンソルの脚は十字型に配置されているが，この文書では表現できないため，一番左の脚から時計回りに $W_{abcd}$としている．

In [1]:
# 4階のテンソルを作成
W = zeros(Int64, 2, 2, 2, 2)

W[2, 1, 1, 1] = 1
W[1, 2, 1, 1] = 1
W[1, 1, 2, 1] = 1
W[1, 1, 1, 2] = 1

println(W)
println(size(W))

[0 1; 1 0;;; 1 0; 0 0;;;; 1 0; 0 0;;; 0 0; 0 0]
(2, 2, 2, 2)


---

### A. 力技でループで和を取る方法（部分和なし）

この場合，和をとる項の数がとても大きくなると予想される．

### 2畳間の場合

![](sec3_2-jo.png)

畳の敷き詰め方 $c$ は，

$$
    c = \sum_{qprs} W_{0 p q 0} W_{q r 0 0} W_{s 0 0 r} W_{0 0 s p}
$$
であるから，単純に和を取ると，

In [5]:
function calc_2_tatami()
    c_tatami = 0
    sum_count = 0
    for p in 1:2
        for q in 1:2
            for r in 1:2
                for s in 1:2
                    c_tatami += W[1, p, q, 1] * W[q, r, 1, 1] * W[s, 1, 1, r] * W[1, 1, s, p]
                    sum_count += 1
                end
            end
        end
    end
    println("c_tatami: $c_tatami")
    println("number of arguments in summation: $sum_count")
end

@time calc_2_tatami()

c_tatami: 2
number of arguments in summation: 16
  0.000319 seconds (176 allocations: 6.297 KiB)


## 3畳間の場合

![](sec3_3-jo.png)

$$
    C = \sum_{pqrstuv} W_{00vp} W_{0ptq} W_{0qr0} W_{rs00} W_{tu0s} W_{v00u}
$$
より

In [6]:
function calc_3_tatami()
    c_tatami = 0
    sum_count = 0
    for p in 1:2
        for q in 1:2
            for r in 1:2
                for s in 1:2
                    for t in 1:2
                        for u in 1:2
                            for v in 1:2
                                c_tatami += W[1,1,v,p] * W[1,p,t,q] * W[1,q,r,1] * W[r,s,1,1] * W[t,u,1,s] * W[v,1,1,u]
                                sum_count += 1
                            end
                        end
                    end
                end
            end
        end
    end
    println("c_tatami: $c_tatami")
    println("number of arguments in summation: $sum_count")
end

@time calc_3_tatami()

c_tatami: 3
number of arguments in summation: 128
  0.006096 seconds (2.17 k allocations: 77.938 KiB, 87.06% compilation time)


### 6畳間の場合

![](sec3_6-jo.png)

$$
    C = \sum_{abcdefghijklmnopq} W_{0ac0} W_{0bda} W_{00eb} W_{cfh0} W_{dgif} W_{e0jg} W_{hkm0} W_{ilnk} W_{j0ol} W_{mp00} W_{nq0p} W_{o00q}
$$

In [7]:
function calc_6_tatami()
    c_tatami = 0
    sum_count = 0
    for a in 1:2; for b in 1:2; for c in 1:2; for d in 1:2
        for e in 1:2; for f in 1:2; for g in 1:2; for h in 1:2
            for i in 1:2; for j in 1:2; for k in 1:2; for l in 1:2
                for m in 1:2; for n in 1:2; for o in 1:2; for p in 1:2
                    for q in 1:2
                        c_tatami += (W[1,a,c,1]*W[1,b,d,a]*W[1,1,e,b]
                            *W[c,f,h,1]*W[d,g,i,f]*W[e,1,j,g]
                                *W[h,k,m,1]*W[i,l,n,k]*W[j,1,o,l]
                                    *W[m,p,1,1]*W[n,q,1,p]*W[o,1,1,q])
                        sum_count += 1
                    end
                end;end;end;end
            end;end;end;end
        end;end;end;end
    end;end;end;end
    println("c_tatami: $c_tatami")
    println("number of arguments in summation: $sum_count")
end

@time calc_6_tatami()


c_tatami: 11
number of arguments in summation: 131072
  0.904771 seconds (3.41 M allocations: 132.001 MiB, 1.26% gc time)


---

## 行列積・行列積関数・転送行列

上の計算方法では，大きくなると計算が一気に破綻してしまう...

e.g. 18畳...36このWの積

部分和を取ることで，計算量を減らす．

## 2畳モデル

![](sec3_part_2-jo.jpg)

1. $W_{0ab0}$と$W_{bc00}$から行列積$T_{ac}$を作る処理

In [19]:
function part_tatami2_T()
    T = zeros(Int64, 2, 2)
    sum_count = 0
    for a in 1:2; for c in 1:2;
        value::Int32 = 0
        for b in 1:2
            value += W[1,a,b,1] * W[b,c,1,1]
            sum_count += 1
        end
        T[a,c] += value
    end;end
    println("T: $T")
    println("sum_count: $sum_count")
    return (T, sum_count)
end
T, sum_count = part_tatami2_T()

T: [1 0; 0 1]
sum_count: 8


([1 0; 0 1], 8)

In [20]:
T[2,2]

1

2. $T_{ac}$と$T_{ac}$で和を取る処理

In [21]:
function part_tatami_2(T::Array{Int64, 2}, sum_count::Int64)
    value = 0
    for a in 1:2;
        for c in 1:2
            value += T[a,c] * T[a,c]
            sum_count += 1
        end
    end
    println("value: $value")
    println("sum_count: $sum_count")
end
part_tatami_2(T, sum_count)

value: 2
sum_count: 12


## 3畳（横）の時

![](sec3_part_3-jo.jpg)

1. $W_{0ad0}$と$W_{dbe0}$と$W_{ec00}$から $T_{abc}$ を作る

- その前に，$W_{0ad0}$と$W_{dbe0}$ から $E_{abe}$を作る
- $E_{abe}$と$W_{ec00}$から$T_{abc}$を作る

In [22]:
function part_tatami3_T()
    sum_count = 0
    E = zeros(Int64, 2, 2, 2)
    # E を作る
    for a in 1:2; for b in 1:2; for e in 1:2
        value = 0
        for d in 1:2
            value += W[1,a,d,1]*W[d,b,e,1]
            sum_count += 1
        end
        E[a,b,e] += value
    end;end;end
    println("E: $E")
    println("sum_count: $sum_count")
    # E から Tを作る
    T = zeros(Int64, 2, 2, 2)
    for a in 1:2; for b in 1:2; for c in 1:2
        value = 0
        for e in 1:2
            value += 
            E[a,b,e] * W[e,c,1,1]
            sum_count += 1
        end
        T[a,b,c] += value
    end;end;end
    println("T: $T")
    println("sum_count: $sum_count")
    return (T, sum_count)
end

T_3, sum_count_T3 = part_tatami3_T()

E: [1 0; 0 1;;; 0 0; 1 0]
sum_count: 16
T: [0 0; 1 0;;; 1 0; 0 1]
sum_count: 32


([0 0; 1 0;;; 1 0; 0 1], 32)

2. $T_{abc}$と$T_{abc}$から全体の個数を計算する

In [23]:
function part_tatami_3(T::Array{Int64, 3}, sum_count::Int64)
    value = 0
    for a in 1:2; for b in 1:2; for c in 1:2
            value += T[a,b,c] * T[a,b,c]
            sum_count += 1
    end; end; end
    println("value: $value")
    println("sum_count: $sum_count")
end
part_tatami_3(T_3, sum_count_T3)

value: 3
sum_count: 40


## 6畳の時

![](sec3_part_6-jo_1.jpg)

1. $W_{0ae0} W_{ebf0} W_{fcg0} W_{gd00}$ から $\Psi_{abcd}$を作りたい
    - まず $W_{0ae0} W_{ebf0}$ から $E_{abf}$ を作る
    - $E_{abf}$と$W_{fcg0}$から$F_{abcg}$を作る
    - $F_{abcg}$と$W_{gd00}$から$\Psi_{abcd}$を作る

In [2]:
function part_tatami6_psi()
    sum_count = 0
    E = zeros(Int64, 2, 2, 2)
    # E を作る
    for a in 1:2; for b in 1:2; for f in 1:2
        value = 0
        for e in 1:2
            value += W[1,a,e,1]*W[e,b,f,1]
            sum_count += 1
        end
        E[a,b,f] += value
    end;end;end
    println("E: $E")
    println("sum_count: $sum_count")
    # E から Fを作る
    F = zeros(Int64, 2, 2, 2, 2)
    for a in 1:2; for b in 1:2; for c in 1:2; for g in 1:2
        value = 0
        for f in 1:2
            value += E[a,b,f] * W[f,c,g,1]
            sum_count += 1
        end
        F[a,b,c,g] += value
    end;end;end;end
    println("F: $F")
    println("sum_count: $sum_count")
    # F と W から psiを作る
    Ψ = zeros(Int64, 2,2,2,2)
    for a in 1:2; for b in 1:2; for c in 1:2; for d in 1:2
        value::Int64 = 0
        for g in 1:2
            value += F[a,b,c,g] * W[g,d,1,1]
            sum_count += 1
        end
        Ψ[a,b,c,d] += value
    end;end;end;end
    println("Ψ: $Ψ")
    println("sum_count: $sum_count")
    return (Ψ, sum_count)
end

Ψ_6, sum_count_Ψ6 = part_tatami6_psi()

E: [1 0; 0 1;;; 0 0; 1 0]
sum_count: 16
F: [0 0; 1 0;;; 1 0; 0 1;;;; 1 0; 0 1;;; 0 0; 0 0]
sum_count: 48
Ψ: [1 0; 0 1;;; 0 0; 0 0;;;; 0 0; 1 0;;; 1 0; 0 1]
sum_count: 80


([1 0; 0 1;;; 0 0; 0 0;;;; 0 0; 1 0;;; 1 0; 0 1], 80)

![](sec3_part_6-jo_2.jpg)

2. 2段の$\Psi'_{pqrs}$を計算する
    - $\Psi_{abcd}$と$W_{fed0}$の積を取って，$D_{abcfe}$を得る
    - $D_{abcfe}$ に $W_{fed0}$ を加える
    - 繰り返す

In [6]:
function part_tatami6_2step(Ψ::Array{Int64, 4}, sum_count::Int64)
    D = zeros(Int64, 2, 2, 2, 2, 2)
    for a in 1:2; for b in 1:2; for c in 1:2; for f in 1:2; for e in 1:2
        value = 0
        for d in 1:2
            value += Ψ[a,b,c,d] * W[f,e,1,d]
            sum_count += 1
        end
        D[a,b,c,f,e] += value
    end;end;end;end;end
    # println("D: $D")

    # D を横に伸ばす
    E = zeros(Int64, 2, 2, 2, 2, 2)
    for a in 1:2; for b in 1:2; for g in 1:2; for h in 1:2; for e in 1:2
        value = 0
        for c in 1:2; for f in 1:2
            value += D[a,b,c,f,e] * W[g,h,f,c]
            sum_count += 1
        end;end
        E[a,b,g,h,e] += value
    end;end;end;end;end

    # E を横に伸ばす
    F = zeros(Int64, 2, 2, 2, 2, 2)
    for a in 1:2; for j in 1:2; for i in 1:2; for h in 1:2; for e in 1:2
        value = 0
        for b in 1:2; for g in 1:2
            value += E[a,b,g,h,e] * W[j,i,g,b]
            sum_count += 1
        end;end
        F[a,j,i,h,e] += value
#    println("F: $F")
    end;end;end;end;end

    # F を横に伸ばす
    G = zeros(Int64, 2, 2, 2, 2)
    for k in 1:2; for i in 1:2; for h in 1:2; for e in 1:2
        value = 0
        for a in 1:2; for j in 1:2
            value += F[a,j,i,h,e] * W[1,k,j,a]
            sum_count += 1
        end;end
        G[k,i,h,e] += value
    end;end;end;end

    println("Ψ': $G")
    println(typeof(G))
    println("sum_count: $sum_count")
    return (G, sum_count)
end

Ψ_6_2step, sum_count_Ψ6_2step = part_tatami6_2step(Ψ_6, sum_count_Ψ6)

Ψ': [5 0; 0 2;;; 0 1; 0 0;;;; 0 0; 1 0;;; 2 0; 0 1]
Array{Int64, 4}
sum_count: 464


([5 0; 0 2;;; 0 1; 0 0;;;; 0 0; 1 0;;; 2 0; 0 1], 464)

![](sec3_part_6-jo_3.jpg)

3. $\Psi'_{abcd}$ と $\Psi_{abcd}$で行列積を取って計算する．

In [8]:
function part_tatami_6(Ψ_1::Array{Int64, 4}, Ψ_2::Array{Int64, 4}, sum_count::Int64)
    value = 0
    for a in 1:2; for b in 1:2; for c in 1:2; for d in 1:2
        value += Ψ_1[a,b,c,d] * Ψ_2[a,b,c,d]
        sum_count += 1
    end;end;end;end
    println("value: $value")
    println("sum_count: $sum_count")
end

part_tatami_6(Ψ_6, Ψ_6_2step, sum_count_Ψ6_2step)

value: 11
sum_count: 480


となり，無事一致し，かつ計算回数が劇的に少なくなっていることがわかる．

## 